In [2]:
import pandas as pd
import Bio.PDB

In [ ]:
#read all results


#map hits to each struc


#cut at thresh of nhits/nstrcuts

#output residue range for each struct


#make core struct folder

#ngl view some strcut vs core eamples


#parse each struct and output core to folder 
parser = Bio.PDB.PDBParser.PDBParser()
for structfile in ranges:
    struct = parser.get_structure(structfile)
    Bio.PDB.Dice.extract()
    struct = 
#clip struct to core

#recalc tree





#